In [ ]:
import json

with open('all_labeled.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

filtered_data = []
for item in data:
    data_info = item.get('data', {})
    images = data_info.get('images')
    # Bỏ các sample có images là [""] hoặc images là None
    if images and not (len(images) == 1 and images[0] == ""):
        image_helpfulness = []
        for ann in item.get('annotations', []):
            for res in ann.get('result', []):
                if res.get('from_name') == 'Image_helpfulness':
                    choices = res.get('value', {}).get('choices', [])
                    for c in choices:
                        try:
                            image_helpfulness.append(int(c))
                        except:
                            pass
        new_item = {
            'images': images,
            'Image_helpfulness': image_helpfulness,
            'product_category': data_info.get('product_category'),
            'product_name': data_info.get('product_name'),
            'id': item.get('id'),  # Lấy id tổng của sample thay vì product_id
        }
        filtered_data.append(new_item)

with open('images.json', 'w', encoding='utf-8') as f:
    json.dump(filtered_data, f, ensure_ascii=False, indent=2)

In [ ]:
# Lấy tất cả các product_category duy nhất
categories = list(set(item['product_category'] for item in filtered_data))

# Tạo mapping từ category sang số
category_to_int = {cat: idx for idx, cat in enumerate(categories)}

# Thêm trường product_category_int vào từng item
for item in filtered_data:
    item['product_category'] = category_to_int[item['product_category']]

# Xem mapping và ví dụ kết quả
print(category_to_int)
print(filtered_data[0]['product_category'], '->', filtered_data[0]['product_category'])
# Lưu thay đổi trực tiếp vào images.json
with open('images.json', 'w', encoding='utf-8') as f_json:
    json.dump(filtered_data, f_json, ensure_ascii=False, indent=2)
with open('category_mapping.md', 'w', encoding='utf-8') as f:
    for cat, idx in category_to_int.items():
        f.write(f"- {cat}: {idx}\n")
print("Đã lưu mapping category vào category_mapping.md")

{'book': 0, 'health&beauty': 1, 'sport': 2, 'food&drink': 3, 'stationery': 4}
3 -> 3
Đã lưu mapping category vào category_mapping.md


In [9]:
import os
import requests
import json

def is_valid_url(url):
    # Kiểm tra url hợp lệ: bắt đầu bằng http hoặc https và có định dạng ảnh hợp lệ
    if not isinstance(url, str):
        return False
    if not url.startswith('http'):
        return False
    ext = os.path.splitext(url)[-1].split('?')[0].lower()
    if ext not in ['.jpg', '.jpeg', '.png', '.webp']:
        return False
    return True

# Đảm bảo thư mục lưu ảnh tồn tại
os.makedirs('downloaded_images', exist_ok=True)

# Đọc dữ liệu từ images.json
with open('images.json', 'r', encoding='utf-8') as f:
    samples = json.load(f)

image_mapping = {}

for sample in samples:
    id_ = sample['id']
    for idx, img_url in enumerate(sample['images']):
        if not is_valid_url(img_url):
            continue
        ext = os.path.splitext(img_url)[-1].split('?')[0]
        if ext.lower() not in ['.jpg', '.jpeg', '.png', '.webp']:
            ext = '.jpg'
        filename = f"{id_}_{idx}{ext}"
        filepath = os.path.join('downloaded_images', filename)
        try:
            response = requests.get(img_url, timeout=10)
            if response.status_code == 200:
                with open(filepath, 'wb') as img_file:
                    img_file.write(response.content)
                if id_ not in image_mapping:
                    image_mapping[id_] = []
                image_mapping[id_].append({'image_file': filename, 'url': img_url})
        except Exception as e:
            print(f"Không tải được ảnh {img_url}: {e}")

# Xuất file mapping dạng gộp theo id
with open('image_mapping.json', 'w', encoding='utf-8') as f:
    json.dump(image_mapping, f, ensure_ascii=False, indent=2)

print("Đã tải xong ảnh và xuất file mapping image_mapping.json")

Đã tải xong ảnh và xuất file mapping image_mapping.json


In [11]:
import os
import json
import random
from PIL import Image, ImageFilter
import numpy as np
import torchvision.transforms as transforms
import torch

# Đọc file image_mapping.json
with open('image_mapping.json', 'r') as f:
    image_mapping = json.load(f)

# Đường dẫn đến thư mục ảnh gốc và thư mục lưu ảnh đã xử lý
input_folder = 'downloaded_images'
output_folder = 'processed_images'

# Tạo thư mục processed_images nếu chưa tồn tại
os.makedirs(output_folder, exist_ok=True)

# Hàm resize với padding để giữ tỷ lệ khung hình
def resize_with_padding(image, target_size=(224, 224)):
    img_width, img_height = image.size
    target_width, target_height = target_size
    
    # Tính tỷ lệ để resize
    ratio = min(target_width / img_width, target_height / img_height)
    new_width = int(img_width * ratio)
    new_height = int(img_height * ratio)
    
    # Resize ảnh giữ tỷ lệ
    image = image.resize((new_width, new_height), Image.Resampling.LANCZOS)
    
    # Tạo ảnh nền đen với kích thước mục tiêu
    padded_image = Image.new('RGB', target_size, (0, 0, 0))  # Màu đen
    offset = ((target_width - new_width) // 2, (target_height - new_height) // 2)
    
    # Dán ảnh đã resize vào giữa ảnh nền
    padded_image.paste(image, offset)
    
    return padded_image

# Định nghĩa transform cơ bản (cho ảnh gốc)
base_transform = transforms.Compose([
    transforms.Lambda(lambda x: resize_with_padding(x)),  # Resize với padding
    transforms.Lambda(lambda x: x.filter(ImageFilter.GaussianBlur(radius=1))),  # Giảm nhiễu
    transforms.ToTensor(),  # Chuyển thành tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),  # Chuẩn hóa
    transforms.ToPILImage()  # Chuyển lại thành PIL Image để lưu
])

# Định nghĩa các transform cho data augmentation
augment_transforms = {
    'flipped': transforms.Compose([
        transforms.Lambda(lambda x: resize_with_padding(x)),
        transforms.Lambda(lambda x: x.filter(ImageFilter.GaussianBlur(radius=1))),
        transforms.RandomHorizontalFlip(p=1.0),  # Lật ngang
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        transforms.ToPILImage()
    ]),
    'rotated': transforms.Compose([
        transforms.Lambda(lambda x: resize_with_padding(x)),
        transforms.Lambda(lambda x: x.filter(ImageFilter.GaussianBlur(radius=1))),
        transforms.RandomRotation(degrees=(-15, 15)),  # Xoay ngẫu nhiên [-15, 15] độ
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        transforms.ToPILImage()
    ]),
    'color': transforms.Compose([
        transforms.Lambda(lambda x: resize_with_padding(x)),
        transforms.Lambda(lambda x: x.filter(ImageFilter.GaussianBlur(radius=1))),
        transforms.ColorJitter(brightness=0.3, contrast=0.3),  # Điều chỉnh độ sáng/tương phản
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        transforms.ToPILImage()
    ])
}

# Tạo file ánh xạ mới
new_mapping = {}

# Xử lý từng ảnh
for item_id, images in image_mapping.items():
    new_mapping[item_id] = []
    for img_info in images:
        img_file = img_info['image_file']
        img_path = os.path.join(input_folder, img_file)
        
        # Kiểm tra file ảnh tồn tại
        if os.path.exists(img_path):
            try:
                # Load ảnh
                img = Image.open(img_path).convert('RGB')
                
                # Xử lý ảnh gốc
                processed_img = base_transform(img)
                output_path = os.path.join(output_folder, img_file)
                processed_img.save(output_path, format='JPEG', quality=95)
                print(f'Processed and saved original: {img_file}')
                
                # Lưu thông tin ảnh gốc vào file ánh xạ mới
                new_mapping[item_id].append({
                    'image_file': img_file,
                    'url': img_info.get('url', ''),
                    'type': 'original'
                })
                
                # Xử lý các phiên bản augment
                for aug_type, aug_transform in augment_transforms.items():
                    aug_img = aug_transform(img)
                    aug_file = f"{os.path.splitext(img_file)[0]}_{aug_type}{os.path.splitext(img_file)[1]}"
                    aug_output_path = os.path.join(output_folder, aug_file)
                    aug_img.save(aug_output_path, format='JPEG', quality=95)
                    print(f'Processed and saved augmented ({aug_type}): {aug_file}')
                    
                    # Lưu thông tin ảnh augment vào file ánh xạ mới
                    new_mapping[item_id].append({
                        'image_file': aug_file,
                        'url': img_info.get('url', ''),
                        'type': aug_type
                    })
                
            except Exception as e:
                print(f'Error processing {img_file}: {str(e)}')
        else:
            print(f'Image not found: {img_file}')

# Lưu file ánh xạ mới
with open('processed_image_mapping.json', 'w') as f:
    json.dump(new_mapping, f, indent=4)
print('Saved new mapping to processed_image_mapping.json')

print('Image preprocessing and augmentation completed.')

Processed and saved original: 211_0.jpg
Processed and saved augmented (flipped): 211_0_flipped.jpg
Processed and saved augmented (rotated): 211_0_rotated.jpg
Processed and saved augmented (color): 211_0_color.jpg
Processed and saved original: 215_0.jpg
Processed and saved augmented (flipped): 215_0_flipped.jpg
Processed and saved augmented (rotated): 215_0_rotated.jpg
Processed and saved augmented (color): 215_0_color.jpg
Processed and saved original: 215_1.jpg
Processed and saved augmented (flipped): 215_1_flipped.jpg
Processed and saved augmented (rotated): 215_1_rotated.jpg
Processed and saved augmented (color): 215_1_color.jpg
Processed and saved original: 215_2.jpg
Processed and saved augmented (flipped): 215_2_flipped.jpg
Processed and saved augmented (rotated): 215_2_rotated.jpg
Processed and saved augmented (color): 215_2_color.jpg
Processed and saved original: 215_1.jpg
Processed and saved augmented (flipped): 215_1_flipped.jpg
Processed and saved augmented (rotated): 215_1_ro